In [ ]:
import tdmclient.notebook
import numpy as np
import sys
#sys.path.append(r'C:\Users\Usuario\OneDrive - epfl.ch\Documents\EPFL\Basics of mobile robotics\PROJET\Github\Mobile_robotics')
import math
import cv2 
import time
import importlib
import filters 
import computer_vision
import path_planning
import obstacle_avoidance

In [ ]:
# Set the speed of the robot
@tdmclient.notebook.sync_var
def set_speed(left, right):
    global motor_left_target, motor_right_target
    motor_left_target = left
    motor_right_target = right

# Get the value of the horizontal distance sensors
@tdmclient.notebook.sync_var
def get_prox():
    obst = prox_horizontal
    return obst

In [ ]:
TRESH_DIST = 10 #Distance Treshold to consider that a goal is reached
CAMERA = 0 # Camera index
SPEED_TO_MMS = 0.3436*1.1 #Conversion factor to convert speed to mm/s
YAW_TO_DEGS = 0.06086*4 #Conversion factor to convert angular speed to deg/s
SPEED_OFFSET = 5 #Speed offset to apply to the right wheel so the robot goes straight
FROM_KHALMAN = 0 #index for plotting purpose
FROM_CAMERA = 1 #index for plotting purpose
GOAL = 2 #index for plotting purpose
NO_OBSTACLE = 0 #state of the robot when no ostacles are detected
OBSTTHRH = 1400 #Threshold to consider that an obstacle is detected
DT = 0.1  #Period of our loop



def main():
         
    try:              
        frame_init = computer_vision.get_image(cap)
        # We need to wait a bit otherwise the image is yellow 
        time.sleep(1)
        
        # Take the first image
        frame_init = computer_vision.get_image(cap)
        
        # Display the image that will be analyzed
        frame_init_crop = cv2.resize(frame_init,None, fx=0.5, fy= 0.5, interpolation = cv2.INTER_CUBIC)
        cv2.imshow('frame', frame_init_crop)
        cv2.waitKey(0)
            
        # Extract vertexes, goals, thymio's start position and orientation from first frame
        start_pos, obst_vertexes, goals_pos, px_to_mm = computer_vision.Init(frame_init)
        
        computer_vision.display_obstacle(frame_init, start_pos, goals_pos, obst_vertexes)
                
        # Compute the optimal path and store it in goal list
        goal_list = path_planning.get_optimal_path(start_pos, goals_pos, obst_vertexes, 
                                                   px_to_mm, draw = True, image = frame_init)
        # Copy goal_list for plot purposes
        optimal_trajectory = goal_list.copy()
        
        # Display the image with all the information on it
        frame_init_crop = cv2.resize(frame_init,None, fx=0.5, fy= 0.5, interpolation = cv2.INTER_CUBIC)
        cv2.imshow('frame', frame_init_crop)
        cv2.waitKey(0)
        
        frame = computer_vision.get_image(cap)
        
        # The estimated state vector at time t-1 in the global reference frame
        # [x_t_minus_1, y_t_minus_1, yaw_t_minus_1] 
        # [mm, mm, degrees] 
        x_est_t_minus_1, hidden_cam, mask_frame = computer_vision.vision(frame, px_to_mm)
        
        # The control input vector at time t-1 in the global reference frame.
        # [v, yaw_rate]
        # [mm/s, degrees/s] 
        u_t_minus_1 = np.array([0 ,0])
         
        # State covariance matrix P_t_minus_1
        P_t_minus_1 = np.array([[1,  0,   0],
                                [0,  1,   0],
                                [0,  0,   1]])
        
        v_l = 0 #Initial left wheel speed
        v_r = 0 #Initial right wheel speed
        state = NO_OBSTACLE #initialize the state of the robot to no obstacle detected
        
        time.sleep(0.1)
        
        while True:

            previous = time.time()
            
            frame = computer_vision.get_image(cap)
            
            # Returns the camera measurement of the robot position and a boolean 
            # indicating if the camera is hidden or not
            obs_vector_z_t, hidden_cam, mask_frame = computer_vision.vision(frame, px_to_mm)
            
            computer_vision.display_obstacle(frame, start_pos, goals_pos, obst_vertexes)
            
            computer_vision.display_pos(frame,obs_vector_z_t, px_to_mm, hidden_cam, FROM_CAMERA)
            
            computer_vision.display_path(frame, optimal_trajectory, px_to_mm)
              
            # Run the Kalman Filter and store the 
            # optimal state and covariance estimates
            optimal_state_estimate_t, covariance_estimate_t = filters.kf(
                obs_vector_z_t, # Most recent sensor measurement
                x_est_t_minus_1, # Our most recent estimate of the state
                u_t_minus_1, # Our most recent control input
                P_t_minus_1, # Our most recent state covariance matrix
                DT,hidden_cam) # Indicator of the camera state
            
            computer_vision.display_pos(frame, optimal_state_estimate_t, px_to_mm, 0, FROM_KHALMAN)
            computer_vision.display_pos(frame, goal_list[0], px_to_mm, 0, GOAL)
            
            # Display frame with all informations on it
            frame = cv2.resize(frame, None, fx=0.5, fy= 0.5, interpolation = cv2.INTER_CUBIC)
            cv2.imshow('frame', frame)
            if cv2.waitKey(1) == ord('q'):
                break
            
            # Display the mask used to detect the position of the robot using opencv
            mask_frame = cv2.resize(mask_frame,None, fx=0.5, fy= 0.5, interpolation = cv2.INTER_CUBIC)
            cv2.imshow('mask frame', mask_frame)
            if cv2.waitKey(1) == ord('q'):
                break
            
            # Compute the distance between the robot and its goal. If this distance < TRESH_DIST, we remove the current goal frome goal_list
            if np.linalg.norm(np.array([optimal_state_estimate_t[0],optimal_state_estimate_t[1]])-np.array(goal_list[0])) < TRESH_DIST:
                goal_list.pop(0)
                # If goal list is empty, it means that the robot has completed the path, we leave the while loop
                if len(goal_list) == 0:
                    break
                
            # Get ready for the next timestep by updating the variable values
            x_est_t_minus_1 = optimal_state_estimate_t
            P_t_minus_1 = covariance_estimate_t
            
            # Check if obstacle in coming
            obst = get_prox()
            if state == NO_OBSTACLE:
                for i in range(len(obst)-2):
                # switch from goal tracking to obst avoidance if obstacle detected
                    if (obst[i] > OBSTTHRH):
                        state = 1
        
            if state == NO_OBSTACLE:
                # Call P controler to obtain the speed of the robot
                v_l,v_r = filters.p_controler(optimal_state_estimate_t,goal_list[0])
            
            else:
                # If we are in local avoidance state
                v, state = obstacle_avoidance.obstacle_avoidance(obst, v_l, v_r)
                v_l = v[0]
                v_r = v[1]
            
            # Conversion of speed and yaw to real world values
            v = (v_l + v_r)*SPEED_TO_MMS/2
            yaw = (v_l-v_r)*YAW_TO_DEGS
            u_t_minus_1 = [v, yaw]; 
            
            # Setting the speed of the robot, correct the fact that the robot doesn't naturally go straight 
            set_speed(int(v_l), int(v_r+SPEED_OFFSET))

            # Sleep the rigth amount of time so the loop runs at a period of DT
            actual = time.time()
            diff = actual-previous
            if diff < DT:
                time.sleep(DT-diff)
            
    finally:
        # Set the speed of the robot to 0 at the end
        set_speed(0, 0)
        # Destroy the frame when we hit a key
        cv2.waitKey(0)
        cv2.destroyAllWindows()



In [ ]:
print("connecting to thymio")
await tdmclient.notebook.start()

In [ ]:
print('Connecting to camera...')
cap=cv2.VideoCapture(CAMERA) # Either 0 or 1, front camera or external cam
print('Camera connected !')

In [ ]:
#get the full quality of the camera
print("setting parameters...")
cap.set(3,1920) 
print("first parameter set")
cap.set(4,1080)
print("second parameter set")

In [ ]:
# Make sure we use the latest version of our libraries
importlib.reload(filters)
importlib.reload(path_planning)
importlib.reload(computer_vision)
importlib.reload(obstacle_avoidance)

In [ ]:
main()

In [ ]:
cap.release()

In [ ]:
await tdmclient.notebook.stop()